In [7]:
pip install langchain_experimental langchain_core

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.messages import HumanMessage

ollama_url = "http://127.0.0.1:11434"

llm = OllamaFunctions(base_url=ollama_url, model="mixtral:8x7b", format="json")

In [3]:
model = llm.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city, and state, e.g. Berlin, Germany. If the location is in the US, include the state, e.g. Seattle, Washington, USA."
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location."
                    }
                },
                "required": ["location", "unit"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

In [4]:
tool = model.invoke([
    HumanMessage(content="What is the weather in San Francisco?")
]).tool_calls

tool

[{'name': 'get_current_weather',
  'args': {'location': 'San Francisco, California, USA', 'unit': 'fahrenheit'},
  'id': 'call_235c14f1074646ffabde9563a8a37f5b'}]

In [5]:
tool = model.invoke([
    HumanMessage(content="What is the weather in Colmar?")
]).tool_calls

tool

[{'name': 'get_current_weather',
  'args': {'location': 'Colmar, France', 'unit': 'celsius'},
  'id': 'call_00712508c6714dfd906df9658f123c67'}]

In [11]:
model = llm.bind_tools(
    tools=[
        {
            "name": "get_stock_price",
            "description": "Get the latest stock price",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "The stock ticker symbol, e.g. AAPL",
                    },
                    "shares": {
                        "type": "int",
                        "description": "The number of shares to get the price for. Default is 1.",
                    }
                },
                "required": ["ticker"],
            },
        }
    ]
)

tool = model.invoke([
    HumanMessage(content="How much is 5 shares of AAPL?"),
]).tool_calls

tool

[{'name': 'get_stock_price',
  'args': {'ticker': 'AAPL', 'shares': 5},
  'id': 'call_920ae854c8cf4abc9ec495747cd253d5'}]

In [6]:
model = llm.bind_tools(
    tools=[
        {
            "name": "get_stock_price",
            "description": "Get the latest stock price",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "The stock ticker symbol, e.g. AAPL",
                    }
                },
                "required": ["location", "unit"],
            },
        },
        {
            "name": "calculate_share_price",
            "description": "Calculate the cost of a given number of shares",
            "parameters": {
                "type": "object",
                "properties": {
                    "number_of_shares": {
                        "type": "int",
                        "description": "The number of shares to buy",
                    },
                    "price": {
                        "type": "float",
                        "description": "The price per share",
                    },
                },
                "required": ["location", "unit"],
            },
        },
        {
            "name": "calculate_share_price_with_a_budget",
            "description": "Calculate the max number of shares one can purchase on a given budget",
            "parameters": {
                "type": "object",
                "properties": {
                    "budget": {
                        "type": "float",
                        "description": "One's budget to spend on shares",
                    },
                    "price": {
                        "type": "float",
                        "description": "The price per share",
                    },
                },
                "required": ["location", "unit"],
            },
        }
    ]
)

tool = model.invoke([
    HumanMessage(content="How many shares of MSFT can I buy with $1000?"),
]).tool_calls

tool

[{'name': 'calculate_share_price_with_a_budget',
  'args': {'budget': 1000, 'price': 23.94},
  'id': 'call_34ea918231d54db198e44d194dc9e9c0'}]